# Tiny A11y Model Fine-tune Notebook
This notebook fine-tunes DeepSeek-Coder on WCAG + MDN accessibility datasets.

Requirements: Free GPU runtime in Colab (T4 recommended).

In [43]:
# Install required packages
!pip install --upgrade pip
!pip install datasets transformers peft huggingface_hub accelerate

In [44]:
# Login to Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()  # Paste your token when prompted

In [45]:
# Load dataset from Hugging Face
from datasets import load_dataset
dataset = load_dataset('younglim/a11y-dataset', split='train')
print(dataset[0])

{'text': "Contributions to this repository are intended to become part of Recommendation-track documents governed by the\n[W3C Patent Policy](https://www.w3.org/Consortium/Patent-Policy-20040205/) and\n[Document License](https://www.w3.org/Consortium/Legal/copyright-documents). To make substantive contributions to specifications, you must either participate\nin the relevant W3C Working Group or make a non-member patent licensing commitment.\n\nIf you are not the sole contributor to a contribution (pull request), please identify all \ncontributors in the pull request comment.\n\nTo add a contributor (other than yourself, that's automatic), mark them one per line as follows:\n\n```\n+@github_username\n```\n\nIf you added a contributor by mistake, you can remove them in a comment with:\n\n```\n-@github_username\n```\n\nIf you are making a pull request on behalf of someone else but you had no part in designing the \nfeature, you can remove yourself with the above syntax.\n\nPublic comments

In [53]:
# Load model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',      # automatically place layers on GPU/CPU
    torch_dtype='auto',     # use float16 where possible
)

# LoRA config for lightweight fine-tuning
peft_config = LoraConfig(r=8, lora_alpha=16, target_modules=['q_proj','v_proj'])
model = get_peft_model(model, peft_config)

In [55]:
# Tokenize dataset
def tokenize(example):
    tokenized = tokenizer(example['text'], truncation=True, padding='max_length', max_length=512)
    tokenized['labels'] = tokenized['input_ids'].copy() # Add labels for training
    return tokenized

dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

In [56]:
# Setup Trainer and TrainingArguments with reduced memory footprint
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=1,  # reduce batch size to fit T4 GPU
    gradient_accumulation_steps=4,  # accumulate grads to simulate larger batch
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,                       # enable mixed precision
    push_to_hub=True,
    hub_model_id='younglim/tiny-a11y-model'
)

trainer = Trainer(model=model, args=training_args, train_dataset=dataset)

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [57]:
# Start training
trainer.train()

Step,Training Loss


TrainOutput(global_step=7, training_loss=5.040900639125279, metrics={'train_runtime': 7.8783, 'train_samples_per_second': 3.3, 'train_steps_per_second': 0.889, 'total_flos': 102394666942464.0, 'train_loss': 5.040900639125279, 'epoch': 1.0})

In [ ]:
# Push the fine-tuned model to Hugging Face Hub
trainer.push_to_hub()

🎉 After running all cells, your fine-tuned Tiny A11y Model will be available at:
[https://huggingface.co/younglim/tiny-a11y-model](https://huggingface.co/younglim/tiny-a11y-model)